# 7.2 Training a basic Convolutional Neural Network for classification of tiny color images.
This examples illustrates a basic CNN in Tensorflow/Keras, trained to classify  32x32 color images of 10 classes. It uses the well-known CIFAR10 dataset. Except the convolutional layers it supports **Max Pooling** layers and **Normalization Layers** that are helpful for the stability of the learning process. 

In [ ]:
# Mount GDrive, change directory and check contents of folder.

import os
from google.colab import drive
from google.colab import files

PROJECT_FOLDER = "/content/gdrive/My Drive/Colab Notebooks/CS345_SP22/6. CNN"

drive.mount('/content/gdrive/')
os.chdir(PROJECT_FOLDER)
print("Current dir: ", os.getcwd())

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from mllib.utils import RandomSeed

# __________ | Settings | __________
IS_PLOTING_DATA         = True
IS_DEBUGABLE            = False
IS_RETRAINING           = True
RandomSeed(2022)

# Hyperparameters
For each training experiment, we define all the model/training hyperparameters inside a Python dictionary.

In [ ]:
CONFIG_CNN = {
                 "ModelName": "MNIST_CNN1"
                ,"CNN.InputShape": [32,32,3]
                ,"CNN.Classes": 10
                ,"CNN.ModuleCount": 6
                ,"CNN.ConvOutputFeatures": [32,32,64,64,128,128]
                ,"CNN.ConvWindows": [ [3,2,True], [3,1,True] ,  [3,1,True], [3,2,True], [3,1,True], [3,1,True] ]
                ,"CNN.PoolWindows": [  None      , None       ,  None      , None      , [3,2]     , None      ]
                ,"CNN.HasBatchNormalization": True
                ,"Training.MaxEpoch": 24
                ,"Training.BatchSize": 128
                ,"Training.LearningRate": 0.1               
            }
                     

We choose the hyperparameter set for the current model training experiment

In [ ]:
CONFIG = CONFIG_CNN

# CIFAR10
The [CIFAR10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) dataset, that dates back to 2009, has become a standard toy dataset to understand the image classification task. It contains 60000 tiny color images of 32x32 resolution for the classes
1. airplane
2. automobile
3. bird
4. cat
5. deer
6. dog
7. frog
8. horse
9. ship
10. truck
                               
It is already splitted into a training set of 50000 images (5000 images per class), while the rest 10000 (1000 images per classe) are used to validate the model

# Dataset loading and previewing
We are using a custom implementation of the dataset that downloads and converts the images into Python pickle files. Then we can display an image and its R, G, B color (feature) maps.

In [ ]:
from datasets.cifar10.dataset import CCIFAR10DataSet

# ... // Create the data objects \\ ...
oDataset = CCIFAR10DataSet()
print("Training samples set shape:", oDataset.TSSamples.shape)
print("Validation samples set shape:", oDataset.VSSamples.shape)

# One hot encoding for the training and validation set labels
nTSLabelsOnehot = keras.utils.to_categorical(oDataset.TSLabels)
nVSLabelsOnehot = keras.utils.to_categorical(oDataset.VSLabels)


import sys
import matplotlib.pyplot as plt

for nIndex, nSample in enumerate(oDataset.TSSamples):
  nLabel = oDataset.TSLabels[nIndex]
  if nIndex == 9: 
    nImage =  nSample.astype(np.uint8) # Show the kitty
    print(nImage.shape)
    print(oDataset.ClassNames[nLabel])
    plt.imshow(nImage[4:22, 0:15, :])
    plt.show()    

  elif nIndex == 30: # Show the toy airplane
    nImage =  nSample.astype(np.uint8)
    print(nImage.shape)
    print(oDataset.ClassNames[nLabel])
    plt.imshow(nImage)
    plt.show()      


    plt.title("Blue")
    plt.imshow(nImage[:,:,0], cmap="Blues")
    plt.show()    
    
    plt.title("Green")
    plt.imshow(nImage[:,:,1], cmap="Greens")
    plt.show()    

    plt.title("Red")
    plt.imshow(nImage[:,:,2], cmap="Reds")
    plt.show()                


# Create the Neural Network model and training algorithm objects


In [ ]:
from models.CNN import CCNNBasic

oNN = CCNNBasic(CONFIG)

# -----------------------------------------------------------------------------------
def LRSchedule(epoch, lr):
    if epoch == 10:
        nNewLR = lr * 0.5
        print("Setting LR to %.5f" % nNewLR)
        return nNewLR
    else:
        return lr
# -----------------------------------------------------------------------------------    

nInitialLearningRate    = CONFIG["Training.LearningRate"]    

oCostFunction   = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
oOptimizer = tf.keras.optimizers.SGD(learning_rate=nInitialLearningRate)
oCallbacks = [tf.keras.callbacks.LearningRateScheduler(LRSchedule)]

Train and evalute the model

In [ ]:
# Train the model
sModelFolderName = CONFIG["ModelName"]
        
if (not os.path.isdir(sModelFolderName)) or IS_RETRAINING:
    oNN.compile(loss=oCostFunction, optimizer=oOptimizer, metrics=["accuracy"])

    if IS_DEBUGABLE:
        oNN.run_eagerly = True
        
    oProcessLog = oNN.fit(  oDataset.TSSamples, nTSLabelsOnehot, batch_size=CONFIG["Training.BatchSize"]  
                            ,epochs=CONFIG["Training.MaxEpoch"]
                            ,validation_data=(oDataset.VSSamples, nVSLabelsOnehot)
                            ,callbacks=oCallbacks
                          )
    oNN.summary()          
    oNN.save(sModelFolderName)      
else:
    # The model is trained and its state is saved (all the trainable parameters are saved). We load the model to recall the samples 
    oNN = keras.models.load_model(sModelFolderName)
    oProcessLog = None
    oNN.summary()  


Inspect the model architecture

# Model Architecture Overview

In [ ]:
oNN.Structure.Print("Model-Structure-%s.csv" % CONFIG["ModelName"])

# Learning Process Overview

In [ ]:
if oProcessLog is not None: # [PYTHON] Checks that object reference is not Null
    # list all data in history
    print("Keys of Keras training process log:", oProcessLog.history.keys())
    
    # Plot the accuracy during the training epochs
    plt.plot(oProcessLog.history['accuracy'])
    plt.plot(oProcessLog.history['val_accuracy'])
    plt.title('CNN Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    # Plot the error during the training epochs
    sCostFunctionNameParts = oCostFunction.name.split("_")                           # [PYTHON]: Splitting string into an array of strings
    sCostFunctionNameParts = [x.capitalize() + " " for x in sCostFunctionNameParts]  # [PYTHON]: List comprehension example 
    sCostFunctionName = " ".join(sCostFunctionNameParts)                             # [PYTHON]: Joining string in a list with the space between them
    
    
    plt.plot(oProcessLog.history['loss'])
    plt.plot(oProcessLog.history['val_loss'])
    plt.title('CNN ' + sCostFunctionName + " Error")
    plt.ylabel('Error')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

# Inference

In [ ]:
nPredictedProbabilities = oNN.predict(oDataset.VSSamples)
nPredictedClassLabels  = np.argmax(nPredictedProbabilities, axis=1)

# Evaluation

In [ ]:
from mllib.evaluation import CEvaluator
from mllib.visualization import CPlotConfusionMatrix

# We create an evaluator object that will produce several metrics
oEvaluator = CEvaluator(oDataset.VSLabels, nPredictedClassLabels)

oEvaluator.PrintConfusionMatrix()
print("Per Class Recall (Accuracy)  :", oEvaluator.Recall)
print("Per Class Precision          :", oEvaluator.Precision)
print("Average Accuracy: %.4f" % oEvaluator.AverageRecall)
print("Average F1 Score: %.4f" % oEvaluator.AverageF1Score)
      
oConfusionMatrixPlot = CPlotConfusionMatrix(oEvaluator.ConfusionMatrix)
oConfusionMatrixPlot.Show()      
